In [317]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [318]:
GROUP1='PSCUC'
GROUP2='PSC'
if (GROUP1 =='UC' and GROUP2=='PSCUC') or (GROUP1 =='PSCUC' and GROUP2=='UC'):
    WORK_DIR="/dsi/sbm/linoym/oracle_cloud/algo_results_uc_pscuc"
elif (GROUP1 =='UC' and GROUP2=='Control') or (GROUP1 =='Control' and GROUP2=='UC'):
    WORK_DIR="/dsi/sbm/linoym/oracle_cloud/algo_results_control_uc"
elif (GROUP1 =='PSCUC' and GROUP2=='PSC') or (GROUP1 =='PSC' and GROUP2=='PSCUC'):
    WORK_DIR="/dsi/sbm/linoym/oracle_cloud/algo_results_psc_pscuc"
else:
    print('No analysis has been performed for those groups.')
print(WORK_DIR)

COMMON_CDR3_PERCENTAGE_THRESHOLD = 80 #%

ADDITIONAL_ANALYSIS_ON_DATA=False
RANDOM_TAGS_TRAIN_CHECK_THE_CODE=False # The flag RANDOM_TAGS_TRAIN_CHECK_THE_CODE creates random labels for the trainig set only. This flag is used to check for data leakage.

/dsi/sbm/linoym/oracle_cloud/algo_results_psc_pscuc


In [ ]:
### 1) The first section check that all the data from SRA selector is exist and that there is no some missing files
### 2) the second section code adds a "label" to the file name using metadata from the SRA selector.
### Add for each output file it's match label - lavel list is foind on "run selector" :  ####
### https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA736966&f=disease_sam_ss%3An%3Auc%2Ccontrol%3Ac&o=acc_s%3Aa ####

### Use this code only when running it for the first time. ###

'''
# Paths
sra_metadata_file = "SraColitisLabel.csv"

# Read the CSV file
df = pd.read_csv(sra_metadata_file, usecols=["Run", "disease"])

# Convert the CSV data into a dictionary for fast lookup
run_to_label = dict(zip(df["Run"], df["disease"]))


### 1) 
# Ensure the column "RUN" exists
if "Run" not in df.columns:
    print("Error: 'RUN' column not found in the CSV file.")
    exit(1)

for run in df["Run"]:
    # Construct the filename
    filename = f"/{WORK_DIR}/CDR3_results_{run}.txt"

    # Check if the file exists
    if os.path.exists(filename):
        continue
    else:
        print(f"File not found: {filename}")

### 2) 
# Process each file in the directory
for file_name in os.listdir(WORK_DIR):
    if file_name.startswith("CDR3_results_SRR") and file_name.endswith(".txt"):
        # Extract the ID from the file name
        file_id = file_name.split("_SRR")[1].split(".txt")[0]  # Extract the "SRRxxxxx" part
        
        # Find the corresponding label
        label = run_to_label.get(f"SRR{file_id}")
        
        if label:
            # Construct the new file name
            new_file_name = f"CDR3_results_SRR{file_id}_{label}.txt"
            
            # Rename the file
            old_file_path = os.path.join(WORK_DIR, file_name)
            new_file_path = os.path.join(WORK_DIR, new_file_name)
            os.rename(old_file_path, new_file_path)
            print(f"Renamed: {file_name} -> {new_file_name}")
        else:
            print(f"No matching label found for: {file_name}")
'''


'\n# Paths\nsra_metadata_file = "SraColitisLabel.csv"\n\n# Read the CSV file\ndf = pd.read_csv(sra_metadata_file, usecols=["Run", "disease"])\n\n# Convert the CSV data into a dictionary for fast lookup\nrun_to_label = dict(zip(df["Run"], df["disease"]))\n\n# Process each file in the directory\nfor file_name in os.listdir(WORK_DIR):\n    if file_name.startswith("CDR3_results_SRR") and file_name.endswith(".txt"):\n        # Extract the ID from the file name\n        file_id = file_name.split("_SRR")[1].split(".txt")[0]  # Extract the "SRRxxxxx" part\n        \n        # Find the corresponding label\n        label = run_to_label.get(f"SRR{file_id}")\n        \n        if label:\n            # Construct the new file name\n            new_file_name = f"CDR3_results_SRR{file_id}_{label}.txt"\n            \n            # Rename the file\n            old_file_path = os.path.join(WORK_DIR, file_name)\n            new_file_path = os.path.join(WORK_DIR, new_file_name)\n            os.rename(old_f

In [320]:
### This code merges all sequences from all files and records their appearance in each file.

# Initialize an empty DataFrame to store the combined results
colitis_all_data_df = pd.DataFrame(columns=["source_file", "CDR3_sequence", "CDR3_copy_number"])

# Iterate through all files in the directory, processing only .txt files.
# For each text file, read the CDR3 sequences and the number of times each sequence appears
for file_name in os.listdir(WORK_DIR):
    if file_name.endswith(".txt"):
        file_path = os.path.join(WORK_DIR, file_name)
        with open(file_path, 'r') as f:
            data = [line.strip().split(',') for line in f.readlines() if ',' in line]
        
        # Store the results in a temporary DataFrame and then combine it with all other results.
        temp_df = pd.DataFrame(data, columns=["CDR3_sequence", "CDR3_copy_number"])
        temp_df["source_file"] = file_name
        colitis_all_data_df = pd.concat([colitis_all_data_df, temp_df], ignore_index=True)

# Ensure the CDR3_copy_number column is numeric
colitis_all_data_df["CDR3_copy_number"] = pd.to_numeric(colitis_all_data_df["CDR3_copy_number"], errors="coerce")
colitis_all_data_df

,source_file,CDR3_sequence,CDR3_copy_number
0,CDR3_results_SRR25927636_PSCUC.txt,CATRSLFF,64
1,CDR3_results_SRR25927636_PSCUC.txt,CVCVCVCVYF,168
2,CDR3_results_SRR25927636_PSCUC.txt,CDWSSDVCSSDL,2316
3,CDR3_results_SRR25927636_PSCUC.txt,CDWSSDVCSSDLF,1598
4,CDR3_results_SRR25927636_PSCUC.txt,PSSEKLFF,2
...,...,...,...
67056,CDR3_results_SRR25927609_PSCUC.txt,CSGGELFF,1
67057,CDR3_results_SRR25927609_PSCUC.txt,GAVSPLTF,1
67058,CDR3_results_SRR25927609_PSCUC.txt,CASQPGGF,1
67059,CDR3_results_SRR25927609_PSCUC.txt,NNNARLMF,1


In [321]:
### This section add label to df 
colitis_all_data_df["label"] = colitis_all_data_df["source_file"].apply(
        lambda x: GROUP1 if f"_{GROUP1}" in x else f"{GROUP2}"
    )


colitis_all_data_df

,source_file,CDR3_sequence,CDR3_copy_number,label
0,CDR3_results_SRR25927636_PSCUC.txt,CATRSLFF,64,PSCUC
1,CDR3_results_SRR25927636_PSCUC.txt,CVCVCVCVYF,168,PSCUC
2,CDR3_results_SRR25927636_PSCUC.txt,CDWSSDVCSSDL,2316,PSCUC
3,CDR3_results_SRR25927636_PSCUC.txt,CDWSSDVCSSDLF,1598,PSCUC
4,CDR3_results_SRR25927636_PSCUC.txt,PSSEKLFF,2,PSCUC
...,...,...,...,...
67056,CDR3_results_SRR25927609_PSCUC.txt,CSGGELFF,1,PSCUC
67057,CDR3_results_SRR25927609_PSCUC.txt,GAVSPLTF,1,PSCUC
67058,CDR3_results_SRR25927609_PSCUC.txt,CASQPGGF,1,PSCUC
67059,CDR3_results_SRR25927609_PSCUC.txt,NNNARLMF,1,PSCUC


In [322]:
# Get unique source files and their labels
file_labels = colitis_all_data_df[["source_file", "label"]].drop_duplicates()

# Stratified split to maintain the ratio of UC and Control files
train_files, test_files = train_test_split(
    file_labels["source_file"], 
    test_size=0.2, 
    # Performs a stratified split to maintain the same ratio of labels files *inside* both train and test sets !!! 
    # For example: if group1 contain 500 files and group2 contain 50 files -> If you perform an 80-20 split, the number of files in each set will be:
    # Group1 (Train: 80% of 500 → 400 files, Test: 20% of 500 → 100 files)
    # Group2 (Train: 80% of 50 → 40 files,Test: 20% of 50 → 10 files)
    # Total in train set --> 440 files and in test set -> 110 files
    stratify=file_labels["label"],
    random_state=42
)

# Create train and test DataFrames
train_df = colitis_all_data_df[colitis_all_data_df["source_file"].isin(train_files)]
test_df = colitis_all_data_df[colitis_all_data_df["source_file"].isin(test_files)]

# Print the number of files in train and test sets
print(f"Number of files in train set: {len(train_files)}")
print(f"Number of files in test set: {len(test_files)}\n")

# Print label distribution inside train and test sets
print("Train set label distribution:")
train_label_counts = train_df.groupby("source_file")["label"].first().value_counts()
print(train_label_counts)

print("\nTest set label distribution:")
test_label_counts = test_df.groupby("source_file")["label"].first().value_counts()
print(test_label_counts)


Number of files in train set: 176
Number of files in test set: 44

Train set label distribution:
label
PSCUC    142
PSC       34
Name: count, dtype: int64

Test set label distribution:
label
PSCUC    35
PSC       9
Name: count, dtype: int64


In [332]:
### This section selects the x percent of common CDR3 sequences from the entire training set.
### The code counts how many times each CDR3 sequence appears in the training data (unpeek the test set!)

# The number of all train files
total_train_files = train_df['source_file'].nunique()
# Count how many times each CDR3 sequence appears in each file.
cdr3_count_in_training_files = train_df.groupby('CDR3_sequence')['source_file'].nunique().reset_index()
# Rename columns for clarity
cdr3_count_in_training_files.columns = ['CDR3_sequence', 'file_count']
# Calculate the percentage
cdr3_count_in_training_files['percentage'] = (cdr3_count_in_training_files['file_count'] / total_train_files) * 100
# Sort by file_count for better readability
cdr3_count_in_training_files.sort_values(by='file_count', ascending=False, inplace=True)


In [333]:
### This section saves only the CDR3 sequences that appear in more than the threshold percentage in the training data.
### Also save only sequnces that strt with C and end with F (cdr3 pattern)
cdr3_sequences_above_threshold_in_training_files = cdr3_count_in_training_files[cdr3_count_in_training_files['percentage'] > COMMON_CDR3_PERCENTAGE_THRESHOLD]
cdr3_sequences_above_threshold_in_training_files = cdr3_sequences_above_threshold_in_training_files[
    cdr3_sequences_above_threshold_in_training_files['CDR3_sequence'].str.startswith('C') &
    cdr3_sequences_above_threshold_in_training_files['CDR3_sequence'].str.endswith('F')
]
cdr3_sequences_above_threshold_in_training_files

,CDR3_sequence,file_count,percentage
7608,CVLFFFFF,176,100.000000
524,CALSDLFF,176,100.000000
1195,CASGVGSF,176,100.000000
1075,CASGLFFF,176,100.000000
5240,CATTPGQFF,176,100.000000
1917,CASSCLSF,176,100.000000
4939,CATALGYF,176,100.000000
7596,CVCVCVCVYF,176,100.000000
5106,CATRSLFF,176,100.000000
5804,CSADLFVF,176,100.000000


In [325]:
############## Preapare data to ML algo ##############
# This section creates a matrix where :
# The rows represent the source file names from the training data
# The columns (features) are the common CDR3 sequences in the training data
# The numeric value in each cell is the copy number of each sequence in the corresponding file.

def prepare_cdr3_data_for_ml(df, cdr3_sequences):
    # Merge with cdr3_sequences
    merged_df = pd.merge(df, cdr3_sequences, on='CDR3_sequence', how='inner')
    
    # Pivot the data
    cdr3_df = merged_df.pivot_table(index='source_file', columns='CDR3_sequence', values='CDR3_copy_number')
    cdr3_df.fillna(0, inplace=True)
    cdr3_df.reset_index(inplace=True)
    
    # Normalize each row to 100%
    ###cdr3_df.iloc[:, 1:-1] = cdr3_df.iloc[:, 1:-1].div(cdr3_df.iloc[:, 1:-1].sum(axis=1), axis=0) * 100
    
    # Clean up column name
    cdr3_df.columns.name = ''
    cdr3_df = cdr3_df.drop('source_file', axis=1)

    # Convert categorical labels to numerical value (0 or 1)
    le =LabelEncoder()
    label_cdr3_df = le.fit_transform(
                                    df.groupby('source_file')['label'].first()
                                     )

    return cdr3_df, label_cdr3_df

# Apply to train and test sets
train_common_cdr3_df,train_labels = prepare_cdr3_data_for_ml(train_df, cdr3_sequences_above_threshold_in_training_files)
test_common_cdr3_df,test_labels = prepare_cdr3_data_for_ml(test_df, cdr3_sequences_above_threshold_in_training_files)

In [326]:
if RANDOM_TAGS_TRAIN_CHECK_THE_CODE is True:
   train_labels = np.random.choice([0, 1], size=train_labels.shape) 
   print("Random tags for test set is created ! (used only for check, otherwise turn off RANDOM_TAGS_TRAIN_CHECK_THE_CODE flag)")

In [327]:
from atom import ATOMClassifier
x_train = train_common_cdr3_df
y_train = train_labels
x_test = test_common_cdr3_df
y_test = test_labels
atom = ATOMClassifier(x_train, x_test, y_train, y_test, verbose=2)

<< ================== ATOM ================== >>
Algorithm task: binary classification.

Dataset stats ==================== >>
Shape: (220, 27)
Train set size: 176
Test set size: 44
-------------------------------------
Memory: 47.65 kB
Scaled: False
Outlier values: 60 (1.3%)



In [328]:
# Usefull commands
#atom.train
#atom.balance(strategy="smote")
#atom.feature_selection(strategy="sfm", solver="xgb", n_features=15) #unused

In [329]:
atom.run(
   models=["LR", "RF", "XGB","AdaB","Bag","BNB","SVM","lSVM"],
   metric="precision",
   n_bootstrap=4,
)


Training ========================= >>
Models: LR, RF, XGB, AdaB, Bag, BNB, SVM, lSVM
Metric: precision


Results for LogisticRegression:
Fit ---------------------------------------------
Train evaluation --> precision: 0.8263
Test evaluation --> precision: 0.7692
Time elapsed: 0.031s
Bootstrap ---------------------------------------
Evaluation --> precision: 0.7733 ± 0.0113
Time elapsed: 0.057s
-------------------------------------------------
Total time: 0.088s


Results for RandomForest:
Fit ---------------------------------------------
Train evaluation --> precision: 1.0
Test evaluation --> precision: 0.7955
Time elapsed: 0.149s
Bootstrap ---------------------------------------
Evaluation --> precision: 0.7881 ± 0.0064
Time elapsed: 0.511s
-------------------------------------------------
Total time: 0.660s


Results for XGBoost:
Fit ---------------------------------------------
Train evaluation --> precision: 1.0
Test evaluation --> precision: 0.8049
Time elapsed: 0.559s
Bootstrap

In [330]:
atom.evaluate(dataset='test')
#atom.evaluate(dataset='train')

,accuracy,average_precision,balanced_accuracy,f1,jaccard,matthews_corrcoef,precision,recall,roc_auc
LR,0.6364,0.7730,0.4413,0.7714,0.6279,-0.1175,0.7714,0.7714,0.4286
RF,0.7955,0.7955,0.5000,0.8861,0.7955,0.0000,0.7955,1.0000,0.5000
XGB,0.7955,0.8043,0.5000,0.8861,0.7955,0.0000,0.7955,1.0000,0.5270
AdaB,0.7273,0.8043,0.4571,0.8421,0.7273,-0.1372,0.7805,0.9143,0.5270
Bag,0.5909,0.7818,0.3714,0.7429,0.5909,-0.2571,0.7429,0.7429,0.4571
BNB,0.7955,0.7955,0.5000,0.8861,0.7955,0.0000,0.7955,1.0000,0.5000
SVM,0.7955,0.7955,0.5000,0.8861,0.7955,0.0000,0.7955,1.0000,0.5000
lSVM,0.6818,0.7730,0.4286,0.8108,0.6818,-0.1816,0.7692,0.8571,0.4286


In [331]:
### Perform some additional analysis ###

def unique_cdr3_by_group(df):
  df_g1 = df[df['source_file'].str.contains(f'_{GROUP1}.txt')]
  df_g2 = df[df['source_file'].str.contains(f'_{GROUP2}.txt')]

  # Get unique CDR3 sequences for each group
  sequences_in_df1 = set(df_g1['CDR3_sequence'])
  sequences_in_df2 = set(df_g2['CDR3_sequence'])

  # Find unique and shared sequences
  seq_only_in_g1 = sequences_in_df1 - sequences_in_df2
  seq_only_in_g2 = sequences_in_df2 - sequences_in_df1
  shared_seqs = sequences_in_df1.intersection(sequences_in_df2)

  print(f"{len(seq_only_in_g1)} CDR3 sequences found only in {GROUP1}.txt files:")
  print(seq_only_in_g1)
  print(f"{len(seq_only_in_g2)} CDR3 sequences found only in {GROUP2}.txt files:")
  print(seq_only_in_g2)
  print(f"{len(shared_seqs)} CDR3 sequences shared to both groups")
  print(seq_only_in_g2)

if ADDITIONAL_ANALYSIS_ON_DATA:
  unique_cdr3_by_group(colitis_all_data_df)
else:
  print(f"ADDITIONAL_ANALYSIS_ON_DATA: {ADDITIONAL_ANALYSIS_ON_DATA}")


ADDITIONAL_ANALYSIS_ON_DATA: False
